Импорт библиотек

In [27]:
import numpy as np
import math
from numpy import matrix
from numpy import linalg

Кодируем алфавит из 32 букв (без буквы "ё"). Для этого создаём словарь, в котором каждую букву кодируем пятью битами.

In [28]:
alp = list('абвгдежзийклмнопрстуфхцчшщъыьэюя')
alp_cod = {}
for i in range(len(alp)):
    str_bin = str(bin(i)[2:])
    if len(str_bin) != 5:
        str_bin = '0' * (5-len(str_bin)) + str_bin
    alp_cod[alp[i]] = str_bin
    
print(alp_cod)

{'а': '00000', 'б': '00001', 'в': '00010', 'г': '00011', 'д': '00100', 'е': '00101', 'ж': '00110', 'з': '00111', 'и': '01000', 'й': '01001', 'к': '01010', 'л': '01011', 'м': '01100', 'н': '01101', 'о': '01110', 'п': '01111', 'р': '10000', 'с': '10001', 'т': '10010', 'у': '10011', 'ф': '10100', 'х': '10101', 'ц': '10110', 'ч': '10111', 'ш': '11000', 'щ': '11001', 'ъ': '11010', 'ы': '11011', 'ь': '11100', 'э': '11101', 'ю': '11110', 'я': '11111'}


Выбираем слово из 4-х букв, которое будем шифровать и получаем список и строку с закодированным словом.

In [29]:
word = 'сидр'
bin_list = [alp_cod[i] for i in word]
bin_word = ''.join(bin_list)

print(bin_list)
print(bin_word)

['10001', '01000', '00100', '10000']
10001010000010010000


Получаем матрицу 4*5 с нашим закодированным словом, в которой каждый столбец соответствует букве.

In [30]:
data_bits = np.zeros((4, 5), dtype=int)
        
for i in range(len(bin_word)):
    bit_value = int(bin_word[i])
    data_bits[i // 5][i % 5] = bit_value
    
print(data_bits)

[[1 0 0 0 1]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]]


Создаем функцию кодирования, в которой используем генераторную матрицу G.

Первые 4 столбца в матрице G - единичная матрица, отвечающая за информацию.

Последние 3 столбца представляют контрольные биты, которые также зависят от информационных битов.

Функция возвращает матрицу 7*5, в которой находится наше закодированное слово с проверочными битами.

In [31]:
def encoding(word):
    
    G = np.array([[1, 0, 0, 0, 1, 1, 1],
                  [0, 1, 0, 0, 1, 1, 0],
                  [0, 0, 1, 0, 0, 1, 1],
                  [0, 0, 0, 1, 1, 0, 1]])
        
    encoding_M = np.dot(np.transpose(G), data_bits) % 2
    
    return encoding_M

p = encoding(word)

print(p)

[[1 0 0 0 1]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [0 1 0 0 1]
 [1 1 1 0 1]
 [0 0 1 0 1]]


Создаём функцию декодирования с использованием матрицы проверки четности H.

Первые 4 столбца будут соответствовать битам данных.

Последние 3 столбца будут содержать единичные векторы, которые представляют биты четности.

Функция возвращает матрицу 4*5, в которой содержится информация о поврежденных битах.

In [32]:
def decoding(p):

    H = np.array([[1, 1, 0, 1, 1, 0, 0],
                  [1, 1, 1, 0, 0, 1, 0],
                  [1, 0, 1, 1, 0, 0, 1]])
    
    decoding_M = np.dot(H, p) % 2

    
    return decoding_M


print(decoding(p))

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


Иммитируем вредоностное вмешательство. Поочередно "портим" 1-4 бита. То есть меняем 1 на 0 и наоборот.

In [33]:
p[0][1] = 0
p[3][2] = 1
p[2][3] = 0
p[5][1] = 1

При вредоносном вмешательстве в 1-4 бита в разных буквах, код показывает ошибку. Но если будут испорчены два бита одной буквы, то ошибку не удастся распознать.

Далее вычисляем позицию ошибки. Для этого находим не нулевой столбец в матрице, которую мы получили после декода(номер этого столбца будет являтся номером столбца ошибки) и находим этот же столбец в матрице Н (номер этого столбца будет являться номером строки ошибки). Испраляем бит и декодируем слово.

In [ ]:
error_syndrome = decoding(p)     

H = np.array([[1, 1, 0, 1, 1, 0, 0],
              [1, 1, 1, 0, 0, 1, 0],
              [1, 0, 1, 1, 0, 0, 1]])


k = [] #Список с номерами столбцов с ошибками
s = [] #Список с ошибками

for i in range(5):
    if not np.array_equal(np.transpose(error_syndrome)[i], np.array([0, 0, 0])):
        k.append(i+1)
        s.append(list(np.transpose(error_syndrome)[i]))


d = [] #Список с номерами строк с ошибок
for j in range(len(s)):
    for y in range(7):
        if np.array_equal(s[j], np.transpose(H)[y]):
            d.append(y-1)


for i in range(len(k)):
    p[k[i]][d[i]] ^= 1

end_word = []        #Превращаем матрицу p в список
for i in range(4):
    end_word += list(p[i])


bin_list_end = [str(i) for i in end_word] # Получаем список, где 1 и 0 являются строками
bin_end = ''.join(bin_list_end)

last_list = []
for i in range(0,20,5):   #Разбиваем строку в список по 5 символов
    last_list += [bin_end[i:i+5]]
    
reversed_alp_code = {value: key for key, value in alp_cod.items()} #Реверсинг словаря, смена ключа и значения

word_end_list = [reversed_alp_code[i] for i in last_list]
word_end = ''.join(word_end_list)

print(word_end)

сидр
